In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load preprocessed train and test datasets
train_df = pd.read_csv("/kaggle/input/opt-dataset/final optimised dataset/train_dataset (1).csv")
test_df = pd.read_csv("/kaggle/input/opt-dataset/final optimised dataset/test.csv")
test_labels_df = pd.read_csv("/kaggle/input/opt-dataset/final optimised dataset/test_labels.csv")

# Merge test set with labels
test_df = test_df.merge(test_labels_df, on="id")

# Identify label columns (assuming they start from 2nd column in test_labels_df)
label_columns = list(test_labels_df.columns[1:])  # Excluding 'id' column

# Convert pandas to Hugging Face dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# Load Deberta tokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")

# First, check if the column exists and what type of data it contains
print(test_ds.column_names)  # Check if "comment_text" exists
print(type(test_ds["comment_text"][0]) if "comment_text" in test_ds.column_names else "Column not found")

# Modified tokenization function with error handling
def tokenize_function(examples):
    # Filter out None/NaN values and ensure all inputs are strings
    texts = []
    for text in examples["comment_text"]:
        if text is None:
            texts.append("")  # Replace None with empty string
        elif not isinstance(text, str):
            texts.append(str(text))  # Convert non-strings to strings
        else:
            texts.append(text)
    
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="np"  # Use numpy arrays instead of PyTorch tensors for now
    )

# Try processing in smaller batches
train_ds = train_ds.map(tokenize_function, batched=True, batch_size=32)
test_ds = test_ds.map(tokenize_function, batched=True, batch_size=32)

# Ensure labels are structured as a list of floats
def format_labels(example):
    example["labels"] = [float(example[col]) for col in label_columns]  # Convert labels to float
    return example

train_ds = train_ds.map(format_labels)
test_ds = test_ds.map(format_labels)

# Remove unnecessary columns (text, id, and individual label columns after structuring)
columns_to_remove = ["comment_text", "id"] + label_columns
train_ds = train_ds.remove_columns(columns_to_remove)
test_ds = test_ds.remove_columns(columns_to_remove)

# Ensure dataset format remains list of floats
def ensure_list_format(example):
    example["labels"] = [float(x) for x in example["labels"]]  # Ensure list of floats
    return example

train_ds = train_ds.map(ensure_list_format)
test_ds = test_ds.map(ensure_list_format)

print("✅ Hatebert Dataset loaded, tokenized & formatted successfully!")

# Save datasets (Optional: Avoid re-running preprocessing)
train_ds.save_to_disk("/kaggle/working/train_ds_hatebert")
test_ds.save_to_disk("/kaggle/working/test_ds_hatebert")

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

['id', 'comment_text', 'severely_toxic', 'moderately_toxic', 'non_toxic']
<class 'str'>


...

Map:   0%|          | 0/298513 [00:00<?, ? examples/s]

Map:   0%|          | 0/127936 [00:00<?, ? examples/s]

✅ Hatebert Dataset loaded, tokenized & formatted successfully!


Saving the dataset (0/1 shards):   0%|          | 0/298513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/127936 [00:00<?, ? examples/s]

In [5]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AdamW
import shutil
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

# Load HateBERT model with sigmoid outputs for multi-label classification
hatebert_model = AutoModelForSequenceClassification.from_pretrained(
    "GroNLP/hateBERT", 
    num_labels=3,  # Matches one-hot encoding format
    problem_type="multi_label_classification"
)

# Increase dropout for better regularization
hatebert_model.config.hidden_dropout_prob = 0.3
hatebert_model.config.attention_probs_dropout_prob = 0.3

# Freeze Transformer Layers, Train Only Classifier
for param in hatebert_model.bert.parameters():
    param.requires_grad = False  
for param in hatebert_model.classifier.parameters():
    param.requires_grad = True  

# Hybrid Loss (Focal + BCE With Logits)
class HybridLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, ce_weight=0.5):
        super(HybridLoss, self).__init__()
        self.focal = FocalLoss(alpha, gamma)
        self.bce = nn.BCEWithLogitsLoss()
        self.ce_weight = ce_weight
        
    def forward(self, inputs, targets):
        return self.ce_weight * self.bce(inputs, targets.float()) + (1 - self.ce_weight) * self.focal(inputs, targets)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        
    def forward(self, inputs, targets):
        BCE_loss = self.bce(inputs, targets.float())
        probs = torch.sigmoid(inputs)
        pt = torch.where(targets.bool(), probs, 1-probs)
        alpha_factor = torch.where(targets.bool(), self.alpha, 1 - self.alpha)
        focal_weight = alpha_factor * (1 - pt).pow(self.gamma)
        return (focal_weight * BCE_loss).mean()

# Training arguments with auto-save best model
training_args = TrainingArguments(
    output_dir="./hatebert_model",
    evaluation_strategy="steps",
    eval_steps=24876,
    save_strategy="steps",
    save_steps=24876,
    save_total_limit=1,
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.1,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_auc",
    greater_is_better=True,
    max_grad_norm=1.0,
    label_smoothing_factor=0.05,
    fp16=True,
    seed=42,
    data_seed=42,
)

# Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    sigmoid_logits = torch.sigmoid(torch.tensor(logits)).numpy()
    predictions = (sigmoid_logits > 0.5).astype(np.int8)
    
    accuracy = accuracy_score(labels, predictions)
    
    try:
        f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
        f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
        auc_scores = [roc_auc_score(labels[:, i], sigmoid_logits[:, i]) for i in range(labels.shape[1]) if len(np.unique(labels[:, i])) > 1]
        mean_auc = np.mean(auc_scores) if auc_scores else 0.0
    except Exception as e:
        print(f"Error in metric calculation: {e}")
        f1_micro, f1_macro, mean_auc = 0, 0, 0
    
    return {"accuracy": accuracy, "f1_micro": f1_micro, "f1_macro": f1_macro, "auc": mean_auc}

# Custom Trainer with Hybrid Loss
class HybridLossTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = HybridLoss(alpha=0.5, gamma=2.0, ce_weight=0.5)
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  
        labels = inputs["labels"].float()  # Keep labels in one-hot format
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Define Trainer
trainer = HybridLossTrainer(
    model=hatebert_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

# Step 1: Train Only Classifier
print("🚀 Training classifier only...")
trainer.train()

# Step 2: Unfreeze last 4 layers
print("🔓 Unfreezing last 4 layers for further fine-tuning...")
for layer in hatebert_model.bert.encoder.layer[-4:]:
    for param in layer.parameters():
        param.requires_grad = True

# Apply Layer-wise Learning Rate
optimizer = AdamW([
    {"params": hatebert_model.bert.encoder.layer[:8].parameters(), "lr": 2e-6},
    {"params": hatebert_model.bert.encoder.layer[8:].parameters(), "lr": 5e-6},
    {"params": hatebert_model.classifier.parameters(), "lr": 1e-5},
], weight_decay=0.1)

trainer.args.num_train_epochs = 2  
trainer.optimizer = optimizer  

# Step 2: Fine-tune last 4 layers
print("🚀 Fine-tuning last 4 layers...")
trainer.train()

# Save best model
best_model_path = "./best_hatebert"
trainer.save_model(best_model_path)
tokenizer.save_pretrained(best_model_path)
shutil.make_archive(best_model_path, 'zip', best_model_path)
print("✅ Best Model Saved!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-4aa5ea5a290e>:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `HybridLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


🚀 Training classifier only...


Step,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Auc
24876,0.274000,0.264844,0.462208,0.595678,0.508486,0.880075
49752,0.239600,0.241750,0.529179,0.649256,0.569643,0.895247
74628,0.233000,0.238685,0.540145,0.657270,0.577680,0.896979


🔓 Unfreezing last 4 layers for further fine-tuning...
🚀 Fine-tuning last 4 layers...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Auc
24876,0.110200,0.055882,0.959988,0.961953,0.961889,0.996332
49752,0.066700,0.045776,0.967257,0.968819,0.968801,0.997309
74628,0.040900,0.043244,0.969055,0.970655,0.970684,0.997459


✅ Best Model Saved!
